In [1]:
import sys
import pandas as pd
import numpy as np
from scipy.optimize import minimize

In [135]:
og_input = pd.read_csv("~/NB-DEA/nb-deainput.csv", index_col=0)

In [136]:
samplenumA = 3
samplenumB = 3

threshold=10

samplestotal = samplenumA+samplenumB

counts = og_input.iloc[:, samplestotal: 2*samplestotal]

avglength = og_input.iloc[:, 2*samplestotal: 3*samplestotal]

counts['Total'] = counts.sum(axis=1)

rows_to_drop = counts[counts['Total'] < threshold].index.tolist()

counts.drop(rows_to_drop, inplace=True)
counts.drop('Total', axis=1, inplace=True)

avglength.drop(rows_to_drop, inplace=True)

In [137]:
avglength

,length.1,length.2,length.3,length.4,length.5,length.6
ENSMUSG00000000001,3213.00,3213.00,3213.00,3213.00,3213.00,3213.00
ENSMUSG00000000028,2094.00,1470.08,1957.50,1811.20,1867.80,2035.80
ENSMUSG00000000049,1141.00,1141.00,1141.00,1141.00,1141.00,1141.00
ENSMUSG00000000056,2548.30,4103.47,3224.95,4068.24,3289.98,3388.22
ENSMUSG00000000058,2289.75,1842.86,1471.19,1814.06,1738.50,1469.17
...,...,...,...,...,...,...
ENSMUSG00000099242,471.00,471.00,471.00,471.00,471.00,471.00
ENSMUSG00000099250,251.00,251.00,251.00,251.00,251.00,251.00
ENSMUSG00000099297,2381.00,2381.00,2381.00,2381.00,2381.00,2381.00
ENSMUSG00000099325,1140.00,1140.00,1140.00,1140.00,1140.00,1140.00


In [138]:
counts

,counts.1,counts.2,counts.3,counts.4,counts.5,counts.6
ENSMUSG00000000001,1212.0,1187.00,1154.00,1031.0,1274.00,848.00
ENSMUSG00000000028,15.0,32.00,10.00,16.0,11.00,13.00
ENSMUSG00000000049,18423.0,17917.00,15619.00,16395.0,16006.00,10709.00
ENSMUSG00000000056,515.0,593.00,606.00,801.0,733.00,553.00
ENSMUSG00000000058,71.0,60.00,44.00,45.0,46.00,41.00
...,...,...,...,...,...,...
ENSMUSG00000099242,12.0,6.00,7.00,8.0,12.00,11.00
ENSMUSG00000099250,120.0,293.00,134.28,44.0,315.50,90.50
ENSMUSG00000099297,11.0,21.00,16.00,13.0,25.00,5.00
ENSMUSG00000099325,3.0,1.50,2.00,5.5,3.50,0.50


In [68]:
means = counts.mean(axis=1)
variances = counts.var(axis=1)

## step1: normalization
want to account for
- sequencing depth
- gene length
- RNA composition

methods (for DE analysis)
- DESeq2's median of ratios
    - counts divided by sample-specific size factors determined by median ratio of gene counts relative to geometric mea n per gene
- EdgeR's trimmed mean of M values (TMM)
    - uses weighted trimmed mean of the log expression of ratios between samples

In [139]:
##using a geometric mean
counts2 = counts.copy()
arr_counts = counts2.to_numpy()
arr_counts[arr_counts == 0] = 1

pseudoref = np.prod(arr_counts, axis=1)**(1/samplestotal)
counts2["Psuedo-Reference"] = pseudoref

size_factor = []
#ratios of each sample to reference
for i in range(samplestotal):
    colname = counts.columns[i]
    counts2['ratio.' + str(i+1)] = counts2[colname]/counts2["Psuedo-Reference"]
    #median of each column is that sample's sizefactor
    median = counts2['ratio.' + str(i+1)].median()
    size_factor.append(median)
    #divide by size factor
    counts[colname] = counts[colname]/median

counts is now normalized!

## Differential Expression

- estimating size factors
- estimating dispersions
- gene-wise dispersion estimates
- mean-dispersion relationship
- final dispersion estimates
- fitting model and testing

In [70]:
size_factor

[1.0851427482015432,
 1.163738465969428,
 1.012377396061459,
 1.0444762783995334,
 1.0406353087207325,
 0.7469075519069984]

We evaluate the variation of expression between groups and compare that to the variation within the groups

In [140]:
stats = counts.copy()
stats = stats.iloc[:, 0:0]
stats["meanA"] = counts.iloc[:, :samplenumA].mean(axis=1)
stats["varA"] = counts.iloc[:, :samplenumA].var(axis=1)
stats["meanB"] = counts.iloc[:, samplenumA:samplestotal].mean(axis=1)
stats["varB"] = counts.iloc[:, samplenumA:samplestotal].mean(axis=1)
stats["BCVa"] = stats["varA"]/(stats["meanA"]**2)
stats["BCVb"] = stats["varB"]/(stats["meanB"]**2)

#### Getting rid of zeros in the count matrix

In [141]:
pseudocount = 1e-6
counts += pseudocount

### Estimating Dispersion

In [143]:
def estimate_dispersion(df):
    def neg_log_likelihood(params, counts):
        mean = np.mean(counts)
        dispersion = params[0]
        n = len(counts)
        log_likelihood = np.sum(counts * np.log(mean) - (counts + 1/dispersion) * np.log(1 + mean * dispersion))
        return -log_likelihood

    dispersions = []
    for gene in df.index:
        counts = df.loc[gene].values
        result = minimize(neg_log_likelihood, x0=[1.0], args=(counts,), bounds=[(1e-10, None)])
        dispersions.append(result.x[0])
    return np.array(dispersions)

In [101]:
dispersion = estimate_dispersion(counts)

In [144]:
stats['dispersion'] = estimate_dispersion(counts)

In [148]:
stats

,meanA,varA,meanB,varB,BCVa,BCVb,dispersion
ENSMUSG00000000001,1092.261163,4049.594395,1115.565905,1115.565905,0.003394,0.000896,1.000000e-10
ENSMUSG00000000028,17.066131,85.502862,14.431416,14.431416,0.293569,0.069293,1.000000e-10
ENSMUSG00000000049,15933.867280,817116.581735,15138.545303,15138.545303,0.003218,0.000066,1.000000e-10
ENSMUSG00000000056,527.582535,4087.424722,737.218356,737.218356,0.014685,0.001356,1.000000e-10
ENSMUSG00000000058,53.483070,123.418154,47.393522,47.393522,0.043147,0.021100,1.000000e-10
...,...,...,...,...,...,...,...
ENSMUSG00000099242,7.709556,9.184519,11.306050,11.306050,0.154525,0.088448,1.000000e-10
ENSMUSG00000099250,164.999199,5769.093578,155.490941,155.490941,0.211906,0.006431,1.000000e-10
ENSMUSG00000099297,14.662197,16.614047,14.388161,14.388161,0.077282,0.069502,1.668245e-10
ENSMUSG00000099325,2.009703,0.545271,3.099518,3.099518,0.135005,0.322631,3.908592e-08


In [145]:
import statsmodels.api as sm
from scipy.stats import chi2
from statsmodels.stats.multitest import multipletests

In [146]:
def fit_negative_binomial(df, num_untreated_replicates, num_treated_replicates, stats):
    results = {}
    group = np.array([0] * num_untreated_replicates + [1] * num_treated_replicates)
    for gene in df.index:
        counts = df.loc[gene].values        
        model = sm.GLM(counts, sm.add_constant(group), family=sm.families.NegativeBinomial(alpha=stats['dispersion'][gene]))
        result = model.fit()
        results[gene] = result
    return results

In [147]:
fit_results = fit_negative_binomial(counts, samplenumA, samplenumB, stats)

In [115]:
def test_differential_expression(fit_results, num_untreated_replicates, num_treated_replicates, stats, df):
    p_values = {}
    log2_fold_changes = {}
    group = np.array([0] * num_untreated_replicates + [1] * num_treated_replicates)
    null_group = np.zeros_like(group)
    for gene, result in fit_results.items():
        counts = df.loc[gene].values
        llf_alt = result.llf
        null_model = sm.GLM(counts, sm.add_constant(null_group), family=sm.families.NegativeBinomial(alpha=stats['dispersion'][gene]))
        null_result = null_model.fit()
        llf_null = null_result.llf
        lr_stat = 2 * (llf_alt - llf_null)
        p_value = chi2.sf(lr_stat, df=1)
        p_values[gene] = p_value
        
        mean_treated = np.mean(counts[group == 1])
        mean_untreated = np.mean(counts[group == 0])
        log2_fold_change = np.log2(mean_treated / mean_untreated)
        log2_fold_changes[gene] = log2_fold_change

    return p_values, log2_fold_changes

In [116]:
p_values, log2_fold_changes = test_differential_expression(fit_results, samplenumA, samplenumB, stats, counts)

In [108]:
rejected, adj_p_values, _, _ = multipletests(list(p_values.values()), method='fdr_bh')

In [109]:
diff_expressed_genes = {gene: p for gene, p in zip(p_values.keys(), adj_p_values) if p < 0.05}
print(len(diff_expressed_genes))

5188


In [120]:
adj_p_values_dict = dict(zip(p_values.keys(), adj_p_values))

In [128]:
results_df = pd.DataFrame({
    'log2FoldChange': log2_fold_changes,
    'pvalue': p_values,
    'padj': adj_p_values_dict
})

In [129]:
results_df.to_csv('differential_expression_results.csv', index=True)